# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Notes
---

## Observations

1. No need to make 10 transactions a day. Just one is enough

1. lema or sema match
    1. Using lema match works fine in both opening and closing positions
    1. Using lema has better results
    1. 144 produces better resuls over higher numbers like 1440 

1. Lema angle in order and close
    1. Using lema angle in order, delays order, but overall profits are high
    1. Using lema angle in close also increases overall profit

1. 2021 vs 2020
    1. 2020 is working with the above setup
    1. 2021 is producing losses in all the above setups

1. Before everything need to find why 2021 is consistently producing negative results in all the setups.
    1. What is that this causing this issue, how can it be addressed with in the same system
    1. less complex method is essential
    
---


## To Test
1. Remove lema match check from closing. This is delaying bad trasactions
1. Test 5 round up on lema match. Transactions are opening in lema match gaps
1. Ideation
    1. Test lema Bollinger bands to understand flow direction
    1. Test overall flow direction

data                = get_date_list(data)
data                = read_data(data)

In [2]:
data['ema_type'] = 'lema'

In [3]:
%%time
data = run_engine(data)

100%|██████████████████████████████████| 74266/74266 [00:27<00:00, 2686.99it/s]

pl   -0.08012
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
71588 2021-12-20 14:45:00         NaN  sema_close -0.00255
71782 2021-12-21 06:55:00         NaN  sema_close -0.00178
71875 2021-12-21 14:40:00         NaN  sema_close -0.00144
72069 2021-12-22 06:50:00         NaN  sema_close -0.00114
72137 2021-12-22 12:30:00         NaN  sema_close -0.00393
72451 2021-12-23 14:40:00         NaN  sema_close -0.00126
72594 2021-12-24 02:35:00         NaN  sema_close -0.00019
72716 2021-12-24 12:45:00         NaN  sema_close -0.00033
73078 2021-12-27 19:00:00         NaN  sema_close -0.00221
73163 2021-12-28 02:05:00         NaN  sema_close -0.00063
73325 2021-12-28 15:35:00         NaN  sema_close -0.00386
73621 2021-12-29 16:15:00         NaN  sema_close -0.00477
73773 2021-12-30 04:55:00         NaN  sema_close -0.00084
73968 2021-12-30 21:10:00         NaN  sema_close -0.00176
74170 2021-12-31 14:00:00         NaN  sema_clo


100%|███████████████████████████████████████| 495/495 [00:00<00:00, 954.47it/s]

Wall time: 1min 45s


In [17]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(x.sort_values())
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1     0.00041
2     0.00353
3     0.00428
4    -0.02306
5     0.01141
6     0.00803
7    -0.01114
8    -0.00011
9    -0.02529
10   -0.00966
11    0.00945
12   -0.04797
Name: pl, dtype: float64
---------------------
month_val
12   -0.04797
9    -0.02529
4    -0.02306
7    -0.01114
10   -0.00966
8    -0.00011
1     0.00041
2     0.00353
3     0.00428
6     0.00803
11    0.00945
5     0.01141
Name: pl, dtype: float64
---------------------
max : 5 : 0.011409999999999995
min : 12 : -0.04797
---------------------
monthly_avg : -0.0067


In [5]:
data['lema_gap_pl_analysis_month'] = 8

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

,index,positive,negative,count_diff,positive_sum,negative_sum,sum_diff
4,0.000,1.0,1,0.0,0.00318,-0.00207,0.00111
0,0.001,1.0,8,-7.0,0.00744,-0.01297,-0.00553
1,0.002,1.0,6,-5.0,0.01135,-0.01170,-0.00035
3,0.003,1.0,1,0.0,0.00325,-0.00004,0.00321
2,0.004,0.0,1,-1.0,0.00000,-0.00107,-0.00107


In [6]:
sys.exit()

SystemExit: 

In [ ]:
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl_type'].count().unstack())
print('----------------------------------------------')
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl'].sum().unstack())

In [ ]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 6]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [10]:
a = np.arange(0,20)
for b in a:
    x = int(round(b/5.0)*5.0)
    print(b, x)

0 0
1 0
2 0
3 5
4 5
5 5
6 5
7 5
8 10
9 10
10 10
11 10
12 10
13 15
14 15
15 15
16 15
17 15
18 20
19 20


In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [19]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
month = 9

# Monthly Chart -------------------
data['plot_start']  = f"{data['input_year']}-{month}-1"

if month == 12:
    data['plot_stop']   = f"{data['input_year']+1}-1-1"
else:
    data['plot_stop']   = f"{data['input_year']}-{month+1}-1"

# Year Chart ----------------------------
# data['plot_start']  = f"{data['input_year']}-1-1"
# data['plot_stop']   = f"{data['input_year']+1}-1-1"
# Year Chart ----------------------------

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
# data['things_to_plot'] = ['candles','sema', 'slema','positions','lema', 'lema_angle', 'lema_max_min', 'close_diff']
# data['things_to_plot'] = ['lema', 'tip', 'candles', 'lema_angle', 'lema_change']
data['things_to_plot'] = ['candles','sema', 'slema','lema', 'lema_match', 'positions', 'lema_change']

plot_graph(data)

generate_result_report(data)

In [ ]:
if np.isnan(data['lema_match']):
    print('yes')
else:
    print('no')

In [ ]:
data['df_ohlc']['lema_match'] = np.nan

set2 = set(data['df_ohlc']['lema_change'][data['df_ohlc']['lema_change'].notnull()].index)

for i in tqdm(np.arange(data['look_back_window_size'], len(data['df_ohlc']))):
    set1 = set(np.arange(i-data['look_back_window_size'],i))
    lema_tips = list(set1 & set2)

    if len(lema_tips) > 0:
        last_lema_tip = lema_tips[-1]
        idx = np.arange(i-data['look_back_window_size'], last_lema_tip+1)
        lema_list = data['df_ohlc']['lema'].loc[idx].round(4)

        if data['df_ohlc']['lema'][i].round(4) in set(lema_list):        
            data['df_ohlc']['lema_match'][i] = data['df_ohlc']['lema'][i]